In [ ]:
from typing import List, Tuple
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from datetime import datetime, timedelta




def one_hot_encode_decorator(func):
    def wrapper(data: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
        encoder = OneHotEncoder(sparse=False)
        categorical_columns_encoded = encoder.fit_transform(data[['Categoria', 'Sub-categoria']])
        return func(data, categorical_columns_encoded)
    return wrapper




def standard_scale_decorator(func):
    def wrapper(data: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
        scaler = StandardScaler()
        numerical_columns_scaled = scaler.fit_transform(data[['Vendas', 'Quantidade', 'Desconto', 'Lucro']])
        return func(data, numerical_columns_scaled)
    return wrapper


@one_hot_encode_decorator
@standard_scale_decorator
def preprocess_data(data: pd.DataFrame, categorical_columns_encoded: np.ndarray, numerical_columns_scaled: np.ndarray) -> pd.DataFrame:
 
    dados = data


def build_and_train_model(X: np.ndarray, y: np.ndarray) -> tf.keras.models.Sequential:
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=256, activation='relu', input_shape=(X.shape[1],)))
    model.add(tf.keras.layers.Dense(units=256, activation='relu'))
    model.add(tf.keras.layers.Dense(units=1, activation='linear'))
    model.compile(optimizer='Adam', loss='mean_squared_error')
    model.fit(X, y, epochs=100, batch_size=64, validation_split=0.2)
    return model


def generate_future_predictions(model: tf.keras.models.Sequential, X: np.ndarray, data: pd.DataFrame) -> Tuple[List[datetime], List[float], List[str], List[str], List[int]]:
    end_date = datetime.now() + timedelta(days=3 * 365)
    date_rng = pd.date_range(start=datetime.now(), end=end_date, freq='M')

    future_dates = []
    future_predictions = []
    cities = []
    states = []
    order_counts = []

    num_predictions = 3 * 12 

    for i in range(num_predictions):
       
        last_X = X[-1].reshape(1, -1)
        last_X = (last_X - X.mean(axis=0)) / X.std(axis=0)

        prediction = model.predict(last_X)

       
        prediction = (prediction * X.std(axis=0)) + X.mean(axis=0)

       
        real_value = prediction[0][0]

        future_predictions.append(real_value)
        future_dates.append(date_rng[i])

       
        last_X = (last_X * X.std(axis=0)) + X.mean(axis=0)
        X = np.concatenate((X, last_X), axis=0)

        cities.append(data['Cidade'].iloc[i])
        states.append(data['Estado'].iloc[i])

        order_count = data[data['Cidade'] == cities[-1]].shape[0]
        order_counts.append(order_count)

    return future_dates, future_predictions, cities, states, order_counts


def find_nearest_date(date, dates):
    return dates[np.argmin(np.abs(dates - date))]






In [ ]:
from collections import defaultdict

def calculate_city_success(city_data):
    total_sales = 0
    total_predictions = 0

    for real_value, prediction, order_count in city_data:
        total_sales += real_value
        total_predictions += prediction

    if total_sales > 0:
        probability = (total_predictions / total_sales) * 100
    else:
        probability = 0

    if probability >= 100:
        recommendation = "Recomendado abrir uma nova loja."
    else:
        recommendation = "Não recomendado abrir uma nova loja imediatamente."

    return probability, recommendation

def group_data_by_state(future_dates, future_predictions, cities, states, order_counts):
    state_data = defaultdict(list)

    for date, prediction, city, state, order_count in zip(future_dates, future_predictions, cities, states, order_counts):
        nearest_date = find_nearest_date(date, data['Data do pedido'])
        real_value = data[data['Data do pedido'] == nearest_date]['Vendas'].values[0]

        state_data[state].append((city, real_value, prediction, order_count))

    return state_data

state_data = group_data_by_state(future_dates, future_predictions, cities, states, order_counts)

for state, cities_data in state_data.items():
    print(f"Estado: {state}")
    for city, real_value, prediction, order_count in cities_data:
        probability, recommendation = calculate_city_success(cities_data)
        print(f"  Cidade: {city} - Probabilidade: {round(probability, 2)}% - {recommendation}")
        print(f"    Valor Real: {real_value:.2f}, Previsão: {round(prediction, 2)}, Pedidos: {order_count}")

In [ ]:
import plotly.graph_objects as go
import pandas as pd

"""
SIMULANDO DF
data = pd.DataFrame({
    'Cidade': ['são paulo', 'brasília', 'santiago', 'goiânia', 'são benedito',
               'pinheiro', 'lençóis paulista', 'abreu e lima', 'natal'],
    'Estado': ['SP', 'DF', 'CH', 'GO', 'CE', 'MA', 'SP', 'PE', 'RN'],
    'Probabilidade': [24.21, 23.80, 23.58, 23.42, 23.31, 23.21, 23.10, 23.00, 22.85],
    'Valor': [12644.66] * 9,
    'Previsao': [3090.55, 3016.98, 2988.76, 2961.22, 2947.77, 2934.50, 2921.37, 2908.40, 2895.59],
    'longitude': [-46.570383, -47.887905, -54.869076, -49.255814, -40.864490, -45.083927, -48.798682, -34.899131, -35.252255],
    'latitude': [-23.567387, -15.794087, -29.191460, -16.673310, -4.044727, -2.522740, -22.597507, -7.904449, -5.750899]
})


normed_values = (data['Valor'] - data['Valor'].min()) / (data['Valor'].max() - data['Valor'].min())

""""
data['text'] = data['Cidade'] + '<br>' + data['Estado'] + '<br>' + \
    'Probabilidade ' + data['Probabilidade'].astype(str) + '<br>' + \
    'Valor ' + data['Valor'].astype(str) + '<br>' + \
    'Previsao ' + data['Previsao'].astype(str)

fig = go.Figure(go.Scattergeo(
    lon=data['longitude'],
    lat=data['latitude'],
    text=data['text'],
    mode='markers',
    marker=dict(
        size=30, 
        opacity=0.8,
        reversescale=True,
        autocolorscale=False,
        symbol='circle',
        line=dict(
            width=1,
            color='rgba(102, 102, 102)'
        ),
        color=normed_values, 
        colorscale='Reds' 
    )
))


for i, row in data.iterrows():
    fig.add_annotation(
        text=row['Cidade'] + ' - ' + row['Estado'],
        x=row['longitude'],
        y=row['latitude'],
        showarrow=False,
    )


fig.update_geos(projection_type='mercator', showland=False) 


fig.update_layout(
    title='Mapa Brasil',
    geo=dict(
        scope='south america',
        showcoastlines=False, 
    )
)


fig.show()